In [1]:
import lightgbm as lgb

In [49]:
# 预测在哪些叶子上：
# y_pred_leaf = gbm.predict(X_test,  num_iteration=gbm.best_iteration, pred_leaf=True)

In [43]:
lgb.__version__

'2.3.0'

In [44]:
import pandas as pd
from sklearn.metrics import mean_squared_error

print('Loading data...')
# load or create your dataset
df_train = pd.read_csv('data/regression/regression.train', header=None, sep='\t')
df_test = pd.read_csv('data/regression/regression.test', header=None, sep='\t')

y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train) # reference (Dataset or None, optional (default=None)) – If this is Dataset for validation, training data should be used as reference.

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

Loading data...
Starting training...
[1]	valid_0's l2: 0.24288	valid_0's l1: 0.491812
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 0.239307	valid_0's l1: 0.48798
[3]	valid_0's l2: 0.235559	valid_0's l1: 0.483905
[4]	valid_0's l2: 0.230771	valid_0's l1: 0.478533
[5]	valid_0's l2: 0.226297	valid_0's l1: 0.47333
[6]	valid_0's l2: 0.223692	valid_0's l1: 0.470079
[7]	valid_0's l2: 0.220941	valid_0's l1: 0.466768
[8]	valid_0's l2: 0.217982	valid_0's l1: 0.462971
[9]	valid_0's l2: 0.215351	valid_0's l1: 0.459474
[10]	valid_0's l2: 0.213064	valid_0's l1: 0.45619
[11]	valid_0's l2: 0.211053	valid_0's l1: 0.4532
[12]	valid_0's l2: 0.209336	valid_0's l1: 0.450526
[13]	valid_0's l2: 0.207492	valid_0's l1: 0.447621
[14]	valid_0's l2: 0.206016	valid_0's l1: 0.445178
[15]	valid_0's l2: 0.204677	valid_0's l1: 0.442718
[16]	valid_0's l2: 0.203224	valid_0's l1: 0.440293
[17]	valid_0's l2: 0.201186	valid_0's l1: 0.437015
[18]	valid_0's l2: 0.199626	valid_0's l1: 0.434725


In [45]:
print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Starting predicting...
The rmse of prediction is: 0.44427172467679765


In [46]:
y_pred_leaf = gbm.predict(X_test,  num_iteration=gbm.best_iteration, pred_leaf=True)

In [47]:
y_pred_leaf.shape # num_boost_round表示树的个数

(500, 20)

In [48]:
y_pred[:10], y_pred_leaf[:10]

(array([0.5947354 , 0.45111956, 0.34806607, 0.50889704, 0.36426725,
        0.33970103, 0.4413845 , 0.36607818, 0.69598061, 0.40820738]),
 array([[29,  2,  8, 30,  2, 15,  9, 24,  2, 20, 17, 30, 19, 26,  7, 14,
         17, 29, 20, 18],
        [ 3,  9, 18,  6,  6,  4, 11, 12, 17, 16,  4, 22,  3,  3,  3, 28,
         28, 15, 13, 14],
        [24, 16, 23,  1,  1,  2,  0,  2, 22,  2,  5,  5,  7,  4,  6,  2,
          2, 10, 30,  1],
        [ 7,  2, 29, 22,  3,  7,  7,  6, 16,  7,  9, 11,  5, 20,  5, 12,
         16,  1, 19, 17],
        [16, 27, 14,  5, 19,  5, 15,  5, 14,  5, 14, 14, 25, 17, 20,  7,
          6,  3,  5,  7],
        [ 3,  2, 25,  3,  3, 14, 22, 21,  3, 15,  9, 11,  3, 23,  3, 26,
         26, 15, 26,  0],
        [16, 12,  3,  5, 19,  5,  4,  5,  1,  5, 14, 14,  1, 17, 20,  7,
          6, 23,  5,  7],
        [16, 18, 13,  5, 19,  5,  7,  5, 15,  5, 14, 14, 13, 17, 20,  7,
          6, 24,  5,  7],
        [29,  8,  7, 30,  2, 15, 25, 30,  4, 14, 19, 23,  6,  3, 12,  

In [39]:
gbm.params

{'boosting_type': 'gbdt',
 'objective': 'regression',
 'metric': {'l1', 'l2'},
 'num_leaves': 31,
 'learning_rate': 0.05,
 'feature_fraction': 0.9,
 'bagging_fraction': 0.8,
 'bagging_freq': 5,
 'verbose': 0}